In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import  warnings
warnings.filterwarnings('ignore')

In [3]:
df1 = pd.read_csv('sih-1.csv')


In [5]:
df2 = pd.read_csv('sih-2.csv')


In [ ]:
df1.head()

In [ ]:
df2.head()

In [11]:
# Convert 'Oct-20' to '10-20'
df2['TEMPERATURE'] = df2['TEMPERATURE'].replace('Oct-20', '10-20')

# Convert temperature ranges to median
def calculate_median(temp):
    if '-' in temp:
        low, high = map(int, temp.split('-'))
        return (low + high) / 2
    return float(temp)

df2['TEMPERATURE'] = df2['TEMPERATURE'].apply(calculate_median)

# Rename 'WATER REQUIREMENT' to 'WATER REQUIREMENT IN LITRES'
df2.rename(columns={'WATER REQUIREMENT': 'WATER REQUIREMENT IN LITRES'}, inplace=True)

In [13]:
df1['TEMPERATURE'] = pd.to_numeric(df1['TEMPERATURE'], errors='coerce')
df2['TEMPERATURE'] = pd.to_numeric(df2['TEMPERATURE'], errors='coerce')


In [15]:
df3 = df2.merge(df1, on='TEMPERATURE', how='left')


In [17]:
# Count unique values in the 'CROP TYPE' column
crop_type_counts = df3["CROP TYPE"].value_counts()

# Print the counts
print("Unique value counts for 'CROP TYPE':")
print(crop_type_counts)

Unique value counts for 'CROP TYPE':
CROP TYPE
WHEAT    416064
Name: count, dtype: int64


In [19]:
df3.head()

,CROP TYPE,SOIL TYPE,REGION,TEMPERATURE,WEATHER CONDITION,WATER REQUIREMENT IN LITRES,SOIL_MOISTURE,HUMIDITY,RAINFALL,STATUS
0,WHEAT,DRY,DESERT,15.0,NORMAL,10.54,26,45,105.008314,ON
1,WHEAT,DRY,DESERT,15.0,NORMAL,10.54,47,28,70.181852,OFF
2,WHEAT,DRY,DESERT,15.0,NORMAL,10.54,2,65,93.350343,OFF
3,WHEAT,DRY,DESERT,15.0,NORMAL,10.54,75,56,72.786242,OFF
4,WHEAT,DRY,DESERT,15.0,NORMAL,10.54,87,54,85.373788,OFF


In [ ]:
# Calculate summary statistics
mean_water_requirement = df3['WATER REQUIREMENT IN LITRES'].mean()
median_water_requirement = df3['WATER REQUIREMENT IN LITRES'].median()
mode_water_requirement = df3['WATER REQUIREMENT IN LITRES'].mode()[0]

# Choose the central tendency measure (mean, median, or mode)
central_value = mean_water_requirement  # You can use median or mode if preferred

# Define the allowed deviation (1% of the central value)
deviation = 0.01 * central_value

# Calculate summary statistics
mean_water_requirement = df3['WATER REQUIREMENT IN LITRES'].mean()
median_water_requirement = df3['WATER REQUIREMENT IN LITRES'].median()
mode_water_requirement = df3['WATER REQUIREMENT IN LITRES'].mode()[0]

mean_soil_moisture = df3['SOIL_MOISTURE'].mean()
median_soil_moisture = df3['SOIL_MOISTURE'].median()
mode_soil_moisture = df3['SOIL_MOISTURE'].mode()[0]

mean_humidity = df3['HUMIDITY'].mean()
median_humidity = df3['HUMIDITY'].median()
mode_humidity = df3['HUMIDITY'].mode()[0]

mean_rainfall = df3['RAINFALL'].mean()
median_rainfall = df3['RAINFALL'].median()
mode_rainfall = df3['RAINFALL'].mode()[0]

# Choose the central tendency measure (mean, median, or mode)
central_value_water = mean_water_requirement  # You can use median or mode if preferred
central_value_soil_moisture = mean_soil_moisture
central_value_humidity = mean_humidity
central_value_rainfall = mean_rainfall

# Define the allowed deviation (1% of the central value)
deviation_water = 0.01 * central_value_water
deviation_soil_moisture = 0.01 * central_value_soil_moisture
deviation_humidity = 0.01 * central_value_humidity
deviation_rainfall = 0.01 * central_value_rainfall


# Define categorical options
soil_types = ['DRY', 'HUMID', 'WET']
regions = ['DESERT', 'SEMI ARID', 'SEMI HUMID', 'HUMID']
temperatures = ['15.0', '25.0', '35.0', '45.0']
weather_conditions = ['NORMAL', 'SUNNY', 'WINDY', 'RAINY']

# Define the number of samples
num_samples = 1000

# Generate synthetic data with tight variations
np.random.seed(42)  # For reproducibility

synthetic_data = {
    'CROP TYPE': ['OTHER'] * num_samples,
    'SOIL TYPE': np.random.choice(soil_types, num_samples),
    'REGION': np.random.choice(regions, num_samples),
    'TEMPERATURE': np.random.choice(temperatures, num_samples),
    'WEATHER CONDITION': np.random.choice(weather_conditions, num_samples),
    'WATER REQUIREMENT IN LITRES': np.random.uniform(
        central_value_water - deviation_water,
        central_value_water + deviation_water,
        num_samples
    ),
    'SOIL_MOISTURE': np.random.uniform(
        central_value_soil_moisture - deviation_soil_moisture,
        central_value_soil_moisture + deviation_soil_moisture,
        num_samples
    ),
    'HUMIDITY': np.random.uniform(
        central_value_humidity - deviation_humidity,
        central_value_humidity + deviation_humidity,
        num_samples
    ),
    'RAINFALL': np.random.uniform(
        central_value_rainfall - deviation_rainfall,
        central_value_rainfall + deviation_rainfall,
        num_samples
    )
}

df_other = pd.DataFrame(synthetic_data)

# Combine the synthetic data with the original data
final_df = pd.concat([df3, df_other], ignore_index=True)
df3=final_df.copy()

In [ ]:
df3.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Initialize label encoders
label_encoders = {}

# Columns to encode
columns_to_encode = ['CROP TYPE', 'SOIL TYPE', 'REGION', 'WEATHER CONDITION']

# Apply label encoding to each categorical column
for column in columns_to_encode:
    # Convert all values to strings
    df3[column] = df3[column].astype(str)
    
    # Initialize and apply label encoder
    le = LabelEncoder()
    df3[column] = le.fit_transform(df3[column])
    label_encoders[column] = le

# Print the label encoders for each column
for column, le in label_encoders.items():
    print(f"\nLabel encoding for {column}:")
    print(dict(zip(le.classes_, le.transform(le.classes_))))

print("\nEncoded DataFrame:")
print(df3)

In [ ]:
df4 = df3.drop(columns=['STATUS'])

In [ ]:
df4.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Compute the correlation matrix
correlation_matrix = df4.corr()

# Print the correlation matrix
print("Correlation Matrix:")
print(correlation_matrix)

# Plot the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# Ensure the columns that should be numeric are actually numeric
numeric_columns = ['TEMPERATURE', 'SOIL_MOISTURE', 'HUMIDITY', 'RAINFALL', 'WATER REQUIREMENT IN LITRES']
df4[numeric_columns] = df4[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Define features (X) and target (y)
X = df4[['CROP TYPE', 'SOIL TYPE', 'REGION', 'TEMPERATURE', 'WEATHER CONDITION', 'SOIL_MOISTURE', 'HUMIDITY', 'RAINFALL']]
y = df4['WATER REQUIREMENT IN LITRES']

# Handle missing values if any
X = X.fillna(X.mean())
y = y.fillna(y.mean())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create and train the Support Vector Machine model
model = SVR(kernel='linear')  # You can adjust the kernel as needed (e.g., 'linear', 'rbf')
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
accuracy = 100 - mape

# Print the results
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}%")
print(f"Accuracy: {accuracy}%")
